This notebook provides tools for analyzing the existing ESG database.
It uses a pre-built version of the database; see `extract_esg_data2`
in `esg_loader.py`

In [6]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import wbgapi as wb
import qgrid
import matplotlib.pyplot as plt
import plotly.express as px
from IPython.core.display import display, HTML

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [36]:
# load and shape raw data
data = pd.read_feather('../data/ESG_wbg.feather')
data = data[(data.date>=2000) & (data.date<2050)]
data['date'] = data['date'].apply(lambda x: int(x))

# arrays of indicators for each pillar
env = data[data.sector=='ENV'].indicatorID.unique()
soc = data[data.sector=='SOC'].indicatorID.unique()
gov = data[data.sector=='GOV'].indicatorID.unique()


# this pivot table provides number of observations (countries) per indicator and year
counts = data.pivot_table(index='indicatorID', columns='date', values='value', aggfunc='count')
counts.columns.rename('Year', inplace=True)
counts.index.rename('Indicators', inplace=True)

# Extract just the indicator names
names = data[['indicatorID', 'indicator']].drop_duplicates().set_index('indicatorID')['indicator']
sectors = data[['indicatorID', 'sector']].drop_duplicates().set_index('indicatorID')['sector']

# We need incomeLevels (from the API) expressed as integers
incomeLevelVals = {'HIC': 3, 'UMC': 2, 'LMC': 1, 'LIC': 0}
economies = wb.economy.DataFrame(skipAggs=True, db=75)
economies['incomeLevelVal'] = economies['incomeLevel'].apply(lambda x: incomeLevelVals[x])

In [13]:
ncountries = data.iso3c.nunique() # set all bars to maximum country number
nseries = data.indicator.nunique()

# here is now to customize the hovertext like you need
# https://stackoverflow.com/questions/64600282

def pillar_figure(limit, title):
    
    df = counts[counts.index.isin(limit)]
    limited_names = names[names.index.isin(limit)]
    indicators = np.repeat(np.array(limited_names), len(counts.columns)).reshape(len(limited_names), len(counts.columns))
    template = 'Year: %{x}<br>Indicator: %{customdata}<br>CETS ID: %{y}<br>Countries: %{z}<extra></extra>'

    fig = px.imshow(df, height=550, zmin=0, zmax=ncountries, title='{} Pillar ({})'.format(title, len(df))).update_layout(
        xaxis=dict(tickmode='array', tickvals=df.columns),
        yaxis=dict(tickmode='array', tickvals=df.index))
    fig.update(data=[dict(customdata=indicators, hovertemplate=template)])
    return(fig)

display(HTML('<h3>Total Countries={}</h3>'.format(ncountries, nseries)))

fig = pillar_figure(env, 'Environment')
fig.show()
fig = pillar_figure(soc, 'Social')
fig.show()
fig = pillar_figure(gov, 'Government')
fig.show()


In [4]:
# Summary table
df = data[['indicatorID', 'indicator', 'sector', 'date']].rename({'date': 'MRY'}, axis=1).groupby('indicatorID').max()
df['MRY_Countries'] = counts.ffill(axis=1).iloc[:,-1]
df['MAX_Countries']  = counts.max(axis=1)

qgrid.show_grid(df[['indicator', 'sector', 'MAX_Countries', 'MRY', 'MRY_Countries']])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [39]:
# interactive tool for analyzing specific indicators
dropdown = list(counts.index)
dropdown.sort()

@interact
def indicator_detail(limit=dropdown):
    display(HTML('<h3>{} - {}</h3>'.format(names[limit], sectors[limit])))
        
    fig = px.bar(counts.loc[limit], range_y=[0, ncountries]).update_layout(
        xaxis=dict(tickmode='array', tickvals=counts.columns),
        yaxis=dict(title="# Countries"),
        showlegend=False)
    fig.update(data=[dict(hovertemplate='Year: %{x}<br># Countries: %{y}<extra></extra>')])
    fig.show()
    
    
    # this gives us a frame of NANs with complete countries and dates
    df = data.pivot_table(index='iso3c', columns='date', values='value', aggfunc='count')\
            .applymap(lambda x: np.nan)
        
    # now update with counts
    df.update(data[data.indicatorID==limit].pivot_table(index='iso3c', columns='date', values='value', aggfunc='count'))
    df = df.apply(lambda x: x + economies.loc[x.name]['incomeLevelVal'], axis=1)
    
    incomeLevels = df.join(economies['incomeLevel'])['incomeLevel']
    incomeLevels = np.repeat(np.array(incomeLevels), len(df.columns)).reshape(len(incomeLevels), len(df.columns))
    customdata = np.dstack((incomeLevels, df.applymap(lambda x: 'no' if np.isnan(x) else 'yes')))
    template = 'Country: %{y} (%{customdata[0]})<br>Year: %{x}<br>Value %{customdata[1]}<extra></extra>'
    
    df = df.join(economies['name'].rename('country')).set_index('country')
    fig = px.imshow(df, height=800).update_layout(
        xaxis=dict(tickmode='array', tickvals=df.columns, title='Year'),
        yaxis=dict(tickmode='array', tickvals=df.index, tickfont={'size': 8}))
    fig.update_layout(yaxis=dict(showticklabels=False, title='Countries'))
    fig.layout.coloraxis.showscale = False
    fig.update(data=[dict(customdata=customdata, hovertemplate=template)])
    
    fig.show()
    
    # jujitsu to make the table display nicely
    display(HTML(wb.series.metadata.get(limit, db=75)._repr_html_()))
    

interactive(children=(Dropdown(description='limit', options=('AG.LND.AGRI.ZS', 'AG.LND.FRST.ZS', 'AG.PRD.FOOD.…